In [1]:
import numpy as np
import torch
import torch.nn as nn

c:\Users\rebal\capstone-project\capstone\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DEHNNLayer(nn.Module):
    def __init__(self, node_in_features, edge_in_features, vn_features, hidden_features):
        super(DEHNNLayer, self).__init__()
        self.node_mlp1 = nn.Sequential(nn.Linear(edge_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, edge_in_features))
        
        self.edge_mlp2 = nn.Sequential(nn.Linear(node_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, node_in_features))
        
        self.edge_mlp3 = nn.Sequential(nn.Linear(2 * node_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, 2 * node_in_features))

        self.node_to_virtual_mlp = nn.Sequential(nn.Linear(node_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, vn_features))
        
        self.virtual_to_higher_virtual_mlp = nn.Sequential(nn.Linear(vn_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, vn_features))
        
        self.higher_virtual_to_virtual_mlp = nn.Sequential(nn.Linear(vn_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, vn_features))
        
        self.virtual_to_node_mlp = nn.Sequential(nn.Linear(vn_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, edge_in_features))


    def forward(self, node_features, edge_features, vn_features, super_vn_features, hypergraph):

        # Node Update
        transformed_edge_features = self.node_mlp1(edge_features)
        updated_node_features = torch.matmul(hypergraph.incidence_matrix, transformed_edge_features)

        # Edge Update
        transformed_node_features = self.edge_mlp2(node_features)
        driver_features = torch.matmul(hypergraph.driver_matrix, transformed_node_features)
        sink_features = torch.matmul(hypergraph.sink_matrix, transformed_node_features)
        updated_edge_features = torch.cat([driver_features, sink_features], dim=1)
        updated_edge_features = self.edge_mlp3(updated_edge_features)
        
        # First Level VN Update
        node_to_virtual_features = self.node_to_virtual_mlp(node_features)
        updated_vn_features = torch.matmul(hypergraph.vn_matrix, node_to_virtual_features)
        updated_vn_features += self.higher_virtual_to_virtual_mlp(super_vn_features)

        # Top Level VN Update
        virtual_to_higher_virtual_features = self.virtual_to_higher_virtual_mlp(vn_features)
        updated_super_vn_features = torch.sum(virtual_to_higher_virtual_features, dim=0)

        # VN to node update
        virtual_to_node_features = self.virtual_to_node_mlp(vn_features)
        propagated_features = torch.matmul(hypergraph.vn_matrix.T, virtual_to_node_features)
        updated_node_features += propagated_features

        return updated_node_features, updated_edge_features, updated_vn_features, updated_super_vn_features


class DEHNN(nn.Module):
    def __init__(self, num_layers, node_in_features, edge_in_features, hidden_features=24):
        super(DEHNN, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        
        # Create multiple layers for DEHNN
        vn_in_features = node_in_features
        for i in range(num_layers):
            self.layers.append(DEHNNLayer(node_in_features, edge_in_features, vn_in_features, hidden_features))
            node_in_features, edge_in_features = edge_in_features, node_in_features
            edge_in_features *= 2

        self.output_layer = nn.Sequential(nn.Linear(node_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, 1))

    def forward(self, node_features, edge_features, vn_features, super_vn_features, hypergraph):
        # Pass through each layer
        for layer in self.layers:
            node_features, edge_features, vn_features, super_vn_features = layer(node_features, edge_features, vn_features, super_vn_features, hypergraph)
        
        # Output prediction for nodes
        output = self.output_layer(node_features)
        return output[:,0]


# Example hypergraph representation class (simplified)
class Hypergraph:
    def __init__(self, incidence_matrix, driver_matrix, sink_matrix, vn_matrix):
        self.incidence_matrix = incidence_matrix
        self.driver_matrix = driver_matrix
        self.sink_matrix = sink_matrix
        self.vn_matrix = vn_matrix


In [100]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

clean_data_dir = '../../data/chips/clean_data/'

n_samples = 13
train_idx = [1, 2, 3, 4, 5, 6, 7, 8]
train_data = []
valid_idx = [11]
valid_data = []
test_idx = [12]
test_data = []

for i in range(1, n_samples+1):
    connectivity = np.load(clean_data_dir + str(i) + '.connectivity.npz')
    incidence_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([connectivity['row'], connectivity['col']])), torch.ones(connectivity['dirs'].shape), dtype=torch.float).to(device)

    drivers = np.load(clean_data_dir + str(i) + '.drivers.npz')
    driver_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([drivers['col'], drivers['row']])), torch.ones(drivers['data'].shape), dtype=torch.float).to(device)

    sinks = np.load(clean_data_dir + str(i) + '.sinks.npz')
    sink_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([sinks['col'], sinks['row']])), torch.ones(sinks['data'].shape), dtype=torch.float).to(device)

    features = np.load(clean_data_dir + str(i) + '.features.npz')
    node_features = features['node_features']
    edge_features = features['net_features']
    # congestion = features['congestion']
    demand = features['demand']

    node_features = torch.tensor(node_features, dtype=torch.float).to(device)
    edge_features = torch.tensor(edge_features, dtype=torch.float).to(device)
    # congestion = torch.tensor(congestion).to(device)
    demand = torch.tensor(demand, dtype=torch.float).to(device)

    num_nodes, num_node_features = node_features.shape
    num_edges, num_edge_features = edge_features.shape

    virtual_nodes = np.load(clean_data_dir + str(i) + '.virtual_nodes.npz')
    vn_rows = virtual_nodes['row']
    vn_cols = virtual_nodes['col']
    vn_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([vn_rows, vn_cols])), torch.ones(len(vn_rows)), dtype=torch.float).to(device)

    num_vn = vn_matrix.shape[0]
    num_vn_features = num_node_features
    vn_features = torch.zeros((num_vn, num_vn_features), dtype=torch.float).to(device)
    super_vn_features = torch.zeros(num_vn_features, dtype=torch.float).to(device)

    hypergraph = Hypergraph(incidence_matrix, driver_matrix, sink_matrix, vn_matrix)

    if i in train_idx:
        train_data.append((node_features, edge_features, vn_features, super_vn_features, hypergraph, demand))
    elif i in valid_idx:
        valid_data.append((node_features, edge_features, vn_features, super_vn_features, hypergraph, demand))
    elif i in test_idx:
        test_data.append((node_features, edge_features, vn_features, super_vn_features, hypergraph, demand))

In [105]:
# Initialize DE-HNN model
model = DEHNN(num_layers=4, node_in_features=num_node_features, edge_in_features=num_edge_features).to(device)
epochs = 200

# Optimizer and Loss Function
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.L1Loss()

# Training Loop (example)
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    for idx in range(len(train_data)):
        node_features, edge_features, vn_features, super_vn_features, hypergraph, congestion = train_data[idx]
        # Forward pass

        output = model(node_features, edge_features, vn_features, super_vn_features, hypergraph)
        # output = output[:,0]
        # print(output)
        
        # Dummy target for illustration (binary labels for each node: 0 for not congested, 1 for congested)
        target = congestion
        
        # print(target)
        # Compute loss
        loss = criterion(output, target)
        
        # Backward pass
        loss.backward()
        optimizer.step()

    # Print loss
    if epoch % 10 == 9:
        model.eval()
        node_features, edge_features, vn_features, super_vn_features, hypergraph, congestion = valid_data[0]
        output = model(node_features, edge_features, vn_features, super_vn_features, hypergraph)
        target = congestion
        valid_loss = criterion(output, target)
        print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {valid_loss.item():.4f}')

Epoch [10/200], Training Loss: 6.7841, Validation Loss: 6.1420
Epoch [20/200], Training Loss: 7.9707, Validation Loss: 6.9138
Epoch [30/200], Training Loss: 7.6455, Validation Loss: 6.6898
Epoch [40/200], Training Loss: 9.0844, Validation Loss: 8.1927
Epoch [50/200], Training Loss: 9.4359, Validation Loss: 8.8962
Epoch [60/200], Training Loss: 6.2220, Validation Loss: 5.6290
Epoch [70/200], Training Loss: 6.7168, Validation Loss: 5.6958
Epoch [80/200], Training Loss: 8.7045, Validation Loss: 6.8922
Epoch [90/200], Training Loss: 8.9685, Validation Loss: 7.6988
Epoch [100/200], Training Loss: 7.2639, Validation Loss: 6.5491
Epoch [110/200], Training Loss: 6.3936, Validation Loss: 5.7060
Epoch [120/200], Training Loss: 6.1860, Validation Loss: 5.5215
Epoch [130/200], Training Loss: 6.5615, Validation Loss: 5.6087
Epoch [140/200], Training Loss: 7.4085, Validation Loss: 5.8965
Epoch [150/200], Training Loss: 8.6132, Validation Loss: 6.9382
Epoch [160/200], Training Loss: 8.8747, Validatio

In [106]:
node_features, edge_features, vn_features, super_vn_features, hypergraph, congestion = test_data[0]
output = model(node_features, edge_features, vn_features, super_vn_features, hypergraph)
target = congestion
real = target.detach().cpu().numpy()
pred = output.detach().cpu().numpy()

# lst = [0, 0, 0, 0]
# for i, j in zip(pred, real):
#     if i >= 0.9 and j >= 0.9:
#         lst[0] += 1
#     elif i < 0.9 and j >= 0.9:
#         lst[1] += 1
#     elif i >= 0.9 and j < 0.9:
#         lst[2] += 1
#     else:
#         lst[3] += 1
# total = sum(lst)
# print(lst)
# print(f'Accuracy: {(lst[0] + lst[3]) / total}')
# print(f'Precision: {lst[0] / (lst[0] + lst[1])}')
# print(f'Recall: {lst[0] / (lst[0] + lst[2])}')

np.mean(np.abs(real - pred))

7.9783225

In [121]:
pred

array([19.873577, 20.288546, 20.128115, ..., 24.603819, 26.225748,
       25.199501], dtype=float32)

In [122]:
real

array([40., 41., 35., ..., 32.,  5.,  9.], dtype=float32)

In [123]:
(real - pred)

array([ 20.126423,  20.711454,  14.871885, ...,   7.396181, -21.225748,
       -16.199501], dtype=float32)